In [3]:
# main libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from torch.nn import functional as F
from torch import nn
from torch import optim
import torchvision
from torchvision import transforms, datasets

In [11]:
# custom libraries
root_main = os.getcwd()
os.chdir("..")
import TorchCommon as TC
os.chdir(root_main)

In [23]:
# main pyperparametrs
valid_size=0.2
batch_size=64
epochs=50
lr=0.1
nrm_mean=0.5
nrm_std=0.5
num_workers=0
root_ds = "D:\GitHub\pytorch-lab\Dataset" # dataset root
root_bm = "D:\GitHub\pytorch-lab\Best_Models" # best models root

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [25]:
# transform
trans=transforms.Compose([
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize((nrm_mean,) , (nrm_std,))
])

In [26]:
#load dataset
train_data=datasets.MNIST(root=root_ds,
                          train=True, transform=trans, download=True)

test_data=datasets.MNIST(root=root_ds,
                          train=False, transform=trans, download=True)

In [27]:
#sampler
num_train = len(train_data)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [28]:
#dataloader
train_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=train_sampler)
valid_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=valid_sampler)
test_loader =DataLoader(test_data,  batch_size= batch_size, num_workers=num_workers, shuffle=True)

In [29]:
# Network
class Net(nn.Module):
    # Assign Intrinsic Properties of Your Neural Network
    def __init__(self):
        super().__init__()
        # Weights of Layer 1th and Layer 2th Are Intrinsic Properties
        self.fc1 = nn.Linear(784, 256, bias=True)
        self.fc2 = nn.Linear(256, 100, bias=True)
        self.fc3 = nn.Linear(100, 10, bias=True)

    # Wiring of Your Network
    def feed_forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = F.relu_(x)
        x = self.fc2(x)
        x = F.relu_(x)
        x = self.fc3(x)
        return x

In [30]:
#create model and set loss function and optimizer
model=Net().to(device)
criterion =nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(), lr=lr)

In [31]:
loss_valid_min=np.Inf
for epoch in range(1,epochs+1):
    loss_train, acc_train = TC.train(model,train_loader, device, optimizer, criterion, epoch)
    loss_valid, acc_valid = TC.valid(model, valid_loader, device, criterion)
    loss_valid_min = TC.save_model(model, optimizer, epoch, root_bm, loss_valid_min, loss_valid)

############################################################
###### Epoch 1 #############################################
############################################################
Loss_Train: 0.49	Acc_Train : 0.84
Loss_Valid: 0.20	Acc_Valid : 0.94

Save Model: YES
############################################################
###### Epoch 2 #############################################
############################################################
Loss_Train: 0.18	Acc_Train : 0.94
Loss_Valid: 0.16	Acc_Valid : 0.95

Save Model: YES
############################################################
###### Epoch 3 #############################################
############################################################
Loss_Train: 0.14	Acc_Train : 0.96
Loss_Valid: 0.14	Acc_Valid : 0.96

Save Model: YES
############################################################
###### Epoch 4 #############################################
############################################################
Loss_Train: 0

KeyboardInterrupt: 

In [32]:
# best model root
root_bm='D:\GitHub\pytorch-lab\Best_Models\model_epoch_6_loss_valid_0.09.pt'

In [33]:
model=Net().to(device)
TC.test(model, test_loader, device, criterion, root_bm)

############################################################
###### TEST INFO ###########################################
############################################################
Best Epoch : 6	Loss_Test : 0.09	Acc_Test : 0.97
############################################################
###### TEST INFO ###########################################
############################################################
